In [2]:
from data import *
import sys
import time
import math
import re
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

user = []
item = []
rating = []

d = Dataset()
d.load_items("allmovies.txt", item)
d.load_users("users.txt", user)
d.load_ratings("ratings.txt", rating)


n_users = len(user)
n_items = len(item)


In [3]:
utility = np.zeros((n_users, n_items))
for r in rating:
    utility[r.user_id - 1][r.item_id - 1] = r.rating

In [4]:
for i in utility:
    print(i)

[ 0.  0.  0. ...,  0.  0.  0.]
[ 9.  0.  0. ...,  0.  0.  0.]
[ 9.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]


In [5]:
movie_genre = []
for movie in item:
    movie_genre.append([movie.action, movie.adventure, movie.animation, movie.comedy,
                        movie.crime, movie.documentary, movie.drama, movie.family,
                        movie.fantasy, movie.history, movie.horror, movie.musical, 
                        movie.mystery, movie.romance, movie.sci_fi, movie.tv_movie, 
                        movie.thriller, movie.war, movie.western])

movie_genre = np.array(movie_genre)
cluster = KMeans(n_clusters=19)
((cluster.fit_predict(movie_genre)))

array([ 8,  4,  4, ...,  2,  1, 12], dtype=int32)

In [6]:
print(len(cluster.fit_predict(movie_genre)))

10131


In [7]:

utility_clustered = []
for i in range(0, n_users):
    average = np.zeros(19)
    # average = [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 0,  0,  0,  0,  0,  0]]
    tmp = []
    for m in range(0, 19):
        tmp.append([])
        # tmp = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for j in range(0, n_items):
        if utility[i][j] != 0:
            tmp[cluster.labels_[j] - 1].append(utility[i][j])
        # tmp = [[rate of movies belonging to cluster 0], [rate of movies belonging to cluster 1], ... [rate of movies belonging to cluster 18]]
    for m in range(0, 19):
        if len(tmp[m]) != 0:
            average[m] = np.mean(tmp[m])
        else:
            average[m] = 0
    utility_clustered.append(average)
utility_clustered = np.array(utility_clustered)
print(utility_clustered)

[[ 0.          0.          0.          0.          5.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          7.33333333  0.          7.          1.          0.          0.
   7.          0.          2.          0.          4.          0.          2.
   0.          0.          0.          6.          7.5       ]
 [ 9.          8.25        0.          0.          3.25        0.          8.
   7.          0.          3.          7.5         9.5         0.          0.
   8.          0.          0.          0.          6.        ]
 [ 0.          0.          0.          0.          8.2         0.          0.
   3.          0.          9.          0.          0.          0.          8.
   0.          0.          0.          0.          7.        ]]


In [8]:
for i in range(0, n_users):
    x = utility_clustered[i]
    num = sum(a for a in x if a > 0)
    den = sum(a > 0 for a in x)
    if den == 0:
        user[i].avg_r = 0
    else:
        user[i].avg_r = num/den 
    print(user[i].avg_r)

5.0
4.87037037037
6.95
7.04


In [9]:
def pcs(x, y):
    num = 0
    den1 = 0
    den2 = 0
    A = utility_clustered[x - 1]
    B = utility_clustered[y - 1]
    num = sum((a - user[x - 1].avg_r) * (b - user[y - 1].avg_r) for a, b in zip(A, B) if a > 0 and b > 0)
    den1 = sum((a - user[x - 1].avg_r) ** 2 for a in A if a > 0)
    den2 = sum((b - user[y - 1].avg_r) ** 2 for b in B if b > 0)
    den = (den1 ** 0.5) * (den2 ** 0.5)
    if den == 0:
        return 0
    else:
        return num / den
pcs_matrix = np.zeros((n_users, n_users))
for i in range(0, n_users):
    for j in range(0, n_users):
        if i!=j:
            pcs_matrix[i][j] = pcs(i + 1, j + 1)


In [10]:
pcs_matrix

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.48449735, -0.61136695],
       [ 0.        ,  0.48449735,  0.        , -0.38350088],
       [ 0.        , -0.61136695, -0.38350088,  0.        ]])

In [11]:
def norm():
    normalize = np.zeros((n_users, 19))
    for i in range(0, n_users):
        for j in range(0, 19):
            if utility_clustered[i][j] != 0:
                normalize[i][j] = utility_clustered[i][j] - user[i].avg_r
            else:
                normalize[i][j] = float('Inf')
    return normalize

def guess(user_id, i_id, top_n):
    similarity = []
    for i in range(0, n_users):
        if i+1 != user_id:
            similarity.append(pcs_matrix[user_id-1][i])
            # a list of pearsons correlation of that user to other users

    temp = norm()
    temp = np.delete(temp, user_id-1, 0)
    combine = zip(similarity,temp)
    sorted_by_similarity = sorted(combine, key=lambda pair:pair[0], reverse=True)
    top = [x for (y,x) in sorted_by_similarity]
    s = 0
#     print(top)
    c = 0
    for i in range(0, top_n):
        if top[i][i_id-1] != float('Inf'):
            s += top[i][i_id-1]
            c += 1
    if c ==0:
        g = user[user_id-1].avg_r
    else:
        g = s/float(c) + user[user_id-1].avg_r
#     print(g)
    return g

In [12]:
utility_copy = np.copy(utility_clustered)
print(utility_copy)

[[ 0.          0.          0.          0.          5.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.        ]
 [ 0.          7.33333333  0.          7.          1.          0.          0.
   7.          0.          2.          0.          4.          0.          2.
   0.          0.          0.          6.          7.5       ]
 [ 9.          8.25        0.          0.          3.25        0.          8.
   7.          0.          3.          7.5         9.5         0.          0.
   8.          0.          0.          0.          6.        ]
 [ 0.          0.          0.          0.          8.2         0.          0.
   3.          0.          9.          0.          0.          0.          8.
   0.          0.          0.          0.          7.        ]]


In [13]:
for i in range(0, n_users):
    for j in range(0, 19):
        if utility_copy[i][j] == 0:
            utility_copy[i][j] = guess(i+1, j+1, 3)

In [14]:
utility_copy

array([[ 7.05      ,  6.88148148,  5.        ,  7.12962963,  5.        ,
         5.        ,  6.05      ,  4.37987654,  5.        ,  3.37987654,
         5.55      ,  5.83981481,  5.        ,  4.04481481,  6.05      ,
         5.        ,  5.        ,  6.12962963,  5.54654321],
       [ 6.92037037,  7.33333333,  4.87037037,  7.        ,  1.        ,
         4.87037037,  5.92037037,  7.        ,  4.87037037,  2.        ,
         5.42037037,  4.        ,  4.87037037,  2.        ,  5.92037037,
         4.87037037,  4.87037037,  6.        ,  7.5       ],
       [ 9.        ,  8.25      ,  6.95      ,  9.07962963,  3.25      ,
         6.95      ,  8.        ,  7.        ,  6.95      ,  3.        ,
         7.5       ,  9.5       ,  6.95      ,  5.99481481,  8.        ,
         6.95      ,  6.95      ,  8.07962963,  6.        ],
       [ 9.09      ,  8.92148148,  7.04      ,  9.16962963,  8.2       ,
         7.04      ,  8.09      ,  3.        ,  7.04      ,  9.        ,
         7.59 